In [1]:
from directory_cobveg_load import definir_diretorio 
project_path = definir_diretorio()

import geopandas as gpd
import os

from utils import(
    get_data_diretorio,
    check_crs,
    save_parquet_excel,
)
from utils.downloads import(
    download_malha_geosampa,
)

# Cobertura Vegetal 2020 (GeoSampa)

In [2]:
data_path = get_data_diretorio(project_path)

## Importar dados 

In [3]:
cache_path = os.path.join(
    data_path, 
    'cache'
)
cache_full_path = os.path.join(
        cache_path, 
        'cobertura_vegetal_concat.parquet'
                )
if os.path.exists(
    cache_full_path
):
    print('Carregando arquivo em cache')
    gdf_cobertura_vegetal = gpd.read_parquet(cache_full_path)
else:
    gdf_cobertura_vegetal = download_malha_geosampa(
        'cobertura_vegetal', 
        cache_path, 
        True
    )

Carregando arquivo em cache


In [4]:
gdf_cobertura_vegetal.shape

(272117, 19)

In [5]:
gdf_cobertura_vegetal.sample(3)

,cd_identif,cd_identi0,cd_control,tx_datum,cd_coorden,cd_coorde0,cd_categor,cd_subcate,cd_subcat0,tx_descric,qt_area_ve,qt_perimet,qt_altura_,qt_desvio_,tx_data_vo,tx_entrega,tx_lote,tx_lote_ve,geometry
154702,154703.0,155061.0,157291.0,"SIRGAS 2000,4 - 23S",323971.17,7388130.07,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",127.47164,49.17270,0.0,0.0,2017,09_2018,009,V2,"POLYGON ((323961.26 7388133.768, 323965.615 73..."
13802,13803.0,13836.0,16130.0,"SIRGAS 2000,4 - 23S",323183.42,7400066.62,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",175.34827,76.73162,0.0,0.0,2017,05_2018,002,V1,"POLYGON ((323186.026 7400076.143, 323186.365 7..."
221798,221799.0,222651.0,348281.0,"SIRGAS 2000,4 - 23S",330679.21,7371131.31,11.0,0.0,0.0,"Baixa cobertura arbórea, arbóreo-arbustiva e o...",281.82091,93.80199,0.0,0.0,2017,10_2018,010,V1,"POLYGON ((330678.053 7371142.81, 330691.942 73..."


## Padronizaçã de colunas

Há uma série de colunas que podem ser importantes ou não. As que eu escolhi apagar são as que constam no dicionário 'drop_cobveg_cols' a seguir:

In [6]:
drop_cobveg_cols ={
    'cd_control',
    'tx_datum',
    'cd_coorden',
    'cd_coorde0',
    'tx_descric',
    'qt_area_ve',
    'qt_perimet',
    'qt_desvio_',
    'tx_data_vo',
    'tx_entrega',
    'tx_lote',
    'tx_lote_ve'    
}

gdf_cobertura_vegetal.drop(
    columns=drop_cobveg_cols,
    axis=1,
    inplace=True
)

In [7]:
rename_cobveg_cols ={
    'cd_identif' : 'cd_cob_veg'
}

gdf_cobertura_vegetal.rename(
    rename_cobveg_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [8]:
gdf_cobertura_vegetal = check_crs(gdf_cobertura_vegetal)

## Salvar GDF

In [9]:
save_parquet_excel(
    gdf_cobertura_vegetal, 
    'malha_cobertura_vegetal',
    data_path,
    data_subpath='assets' 
)